In [73]:
import numpy as np
import pandas as pd
import jieba
import textdistance as td
from tqdm import tqdm
from gensim import corpora,models,similarities
from gensim.test.utils import common_texts
from gensim.models import Word2Vec,TfidfModel
from gensim import corpora

In [82]:
#文本处理，有些可能用不到
import re
import string
import jieba
with open("baidu_stopwords.txt",encoding="utf-8") as f:
    stopword_list=f.readlines()

def tokenize_text(text):
    tokens=jieba.cut(text)
    tokens=[token.strip() for token in tokens]
    return tokens

def remove_special_characters(text):
    tokens=tokenize_text(text)
    pattern=re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens=filter(None,[pattern.sub('',token) for token in tokens])
    filtered_text=''.join(filtered_tokens)
    return filtered_text

#去除停用词
def remove_stopwords(text):
    tokens=tokenize_text(text)
    filtered_tokens=[token for token in tokens if token not in stopword_list]
    filtered_text=''.join(filtered_tokens)
    return filtered_text

def normalize_corpus(corpus,tokenize=False):
    normalize_corpus=[]
    for text in corpus:
        text=remove_special_characters(text)
        text=remove_stopwords(text)
        if tokenize:
            normalize_corpus.append(tokenize_text(text))
        else:
            normalize_corpus.append(text)
    return normalize_corpus

In [89]:
#用于提取tfidf内积作为对比相似度
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_extractor(corpus,ngram_range=(1,1)):
    vectorizer=TfidfVectorizer(min_df=1,
                              norm='l2',
                              smooth_idf=True,
                              use_idf=True,
                              ngram_range=ngram_range)
    features=vectorizer.fit_transform(corpus)
    return vectorizer,features


def participle_text(text):
    words_list=jieba.lcut(text)
    return ' '.join(words_list)


#得到tfidf特征向量的内积距离
def get_tfidfvec_dis(tfidf_vectorizer,text1,text2):
    fit_text1=tfidf_vectorizer.transform([participle_text(text1)])
    fit_text2=tfidf_vectorizer.transform([participle_text(text2)])
    vec1=fit_text1.toarray()[0]
    vec2=fit_text2.toarray()[0]
    return np.dot(vec1,vec2)

def tfidfvec_dis_list(tfidf_vectorizer,df):
    dis_list=[]
    for text1,text2 in zip(df['text1'],df['text2']):
        dis_list.append(get_tfidfvec_dis(tfidf_vectorizer,text1,text2))
    return dis_list

In [74]:
train_raw = pd.read_csv('paws-x-zh/paws-x-zh/train.tsv', sep='\t',names=['text_a', 'text_b', 'label'])
test_raw = pd.read_csv('paws-x-zh/paws-x-zh/test.tsv', sep='\t',names=['text_a', 'text_b', 'label'])
test_raw['label'] = -1
train_raw = train_raw.dropna()
test_raw = test_raw.dropna()

In [84]:
A_corpus = normalize_corpus(train_raw['text_a'].values.tolist())
B_corpus = normalize_corpus(train_raw['text_b'].values.tolist())
corpusall = A_corpus+B_corpus
tokenized_corpus=[' '.join(jieba.lcut(text)) for text in corpusall]

In [90]:
tfidf_vectorizer,tfidf_train_features=tfidf_extractor(tokenized_corpus)

In [95]:
get_tfidfvec_dis(tfidf_vectorizer,'1975年的NBA赛季 - 76赛季是全美篮球协会的第30个赛季。','1975-76赛季的全国篮球协会是NBA的第30个赛季。')

0.8752516213468617

In [9]:
train_raw['len_a'] = train_raw['text_a'].apply(lambda x:len(x))
p = np.percentile(train_raw['len_a'].tolist(), [50,75,90,97.5]) # return 50th percentile, e.g median.
p

array([42., 55., 69., 88.])

In [100]:
def cut(content):
    try:
        seg_list = jieba.lcut(content, cut_all=True)
    except AttributeError as ex:
        print(content)
        raise ex
    return seg_list

#text1和text2长度差
def len_diff(text1,text2):
    return abs(len(text1)-len(text2))
def both_num(list1,list2):
    dict1,dict2,ans={},{},0
    for i in list1:
        dict1[i]=list1.count(i)
    for i in list2:
        dict2[i]=list2.count(i)
    for k,v in dict1.items():
        tmp=0 if k not in list2 else dict2[k]
        ans+=min(v,tmp)
    return ans

def both_words_num(text1,text2):
    a,b=jieba.lcut(text1),jieba.lcut(text2)
    return both_num(a,b)

# text1和text2共有字符的个数
def both_chars_num(text1,text2):
    a,b=[i for i in text1],[i for i in text2]
    return both_num(a,b)


#text1与text2共有单词的个数 / text1字符个数
def both_words_divideby_char1(text1,text2):
    return both_words_num(text1,text2)/len(text1)

#text1与text2共有单词的个数 / text2字符个数
def both_words_divideby_char2(text1,text2):
    return both_words_num(text1,text2)/len(text2)

def editDistance(word1: str, word2: str) -> int:
    n1 = len(word1)
    n2 = len(word2)
    dp = [[0] * (n2 + 1) for _ in range(n1 + 1)]
       
       # init
    for j in range(1, n2 + 1):
        dp[0][j] = j
    for i in range(1, n1 + 1):
        dp[i][0] = i
       
    for i in range(1, n1 + 1):
        for j in range(1, n2 + 1):
            if word1[i-1] == word2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1] ) + 1      
    return dp[-1][-1]
def data_anaysis(df):
    # 距离
    df['edit_dist'] = df.apply(lambda row: editDistance(row['text_a'], row['text_b']), axis=1)
    df['jaccard_dist'] = df.apply(lambda row: td.jaccard(row['text_a'], row['text_b']), axis=1)
    df['tversky_dist'] = df.apply(lambda row: td.tversky(row['text_a'], row['text_b']), axis=1)
    df['jaro_dist'] = df.apply(lambda row: td.jaro(row['text_a'], row['text_b']), axis=1)
    df['tfidf_dist'] = df.apply(lambda row: get_tfidfvec_dis(tfidf_vectorizer,row['text_a'], row['text_b']), axis=1)
    # 长度差
    df['len_diff'] = df.apply(lambda row: len_diff(row['text_a'], row['text_b']), axis=1)
    # 共有单词的个数
    df['both_words_num'] = df.apply(lambda row: both_words_num(row['text_a'], row['text_b']), axis=1)
    df['both_chars_num'] = df.apply(lambda row: both_chars_num(row['text_a'], row['text_b']), axis=1)
    # 单词个数比
    df['both_words_divideby_char1'] = df.apply(lambda row: both_words_divideby_char1(row['text_a'], row['text_b']), axis=1)
    df['both_words_divideby_char2'] = df.apply(lambda row: both_words_divideby_char2(row['text_a'], row['text_b']), axis=1)
    
    return df

In [101]:
train = data_anaysis(train_raw)
test = data_anaysis(test_raw)

In [102]:
train

,text_a,text_b,label,edit_dist,jaccard_dist,tversky_dist,jaro_dist,tfidf_dist,len_diff,both_words_num,both_chars_num,both_words_divideby_char1,both_words_divideby_char2
0,1560年10月，他在巴黎秘密会见了英国大使Nicolas Throckmorton，要求他...,1560年10月，他在巴黎秘密会见了英国大使尼古拉斯·斯罗克莫顿，并要求他通过英格兰返回苏格...,0,28,0.486111,0.486111,0.760901,0.762949,5,20,35,0.357143,0.392157
1,1975年的NBA赛季 - 76赛季是全美篮球协会的第30个赛季。,1975-76赛季的全国篮球协会是NBA的第30个赛季。,1,16,0.771429,0.771429,0.796011,0.875252,6,15,27,0.441176,0.535714
2,还有具体的讨论，公众形象辩论和项目讨论。,还有公开讨论，特定档案讨论和项目讨论。,0,8,0.500000,0.500000,0.675506,0.620696,1,7,13,0.350000,0.368421
3,当可以保持相当的流速时，结果很高。,当可以保持可比较的流速时，结果很高。,1,3,0.750000,0.750000,0.905229,0.788200,1,11,15,0.647059,0.611111
4,它是Akmola地区Zerendi区的所在地。,它是Akmola地区Zerendi区的所在地。,1,0,1.000000,1.000000,1.000000,1.000000,0,9,23,0.391304,0.391304
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49396,``我们的学校是精神和精神，热爱（时间路径）是我们的第一承诺''。,``我们的学校属于时间和精神，对Rehit的爱（精神之路）是我们的第一承诺。 “”,0,19,0.608696,0.608696,0.772375,0.760011,8,18,28,0.545455,0.439024
49397,她于6月24日在科克，并于7月8日抵达。,她于6月24日在科克，并于7月8日抵达唐斯。,1,2,0.909091,0.909091,0.969697,0.786773,2,16,20,0.800000,0.727273
49398,Cornelia Stuyvesant Vanderbilt（George和Edith Va...,John John F. A. Cecil（George和Cornelia Stuyvesa...,0,68,0.891667,0.891667,0.748008,0.827630,7,38,107,0.345455,0.324786
49399,第三季于2010年6月7日首播，第四季是混合情侣竞赛系统。,第四季于2010年6月7日首播。就像第三季一样，比赛系统是混合情侣。,0,16,0.800000,0.800000,0.760236,0.849129,5,16,28,0.551724,0.470588


In [103]:
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb

# 建模
fretures = ['len_diff','both_words_num','both_chars_num','both_words_divideby_char1','both_words_divideby_char2','tfidf_dist','jaccard_dist','tversky_dist','jaro_dist','edit_dist']
X = train[fretures]
y = train['label']
test_features = test[fretures]

model = lgb.LGBMClassifier(num_leaves=128,
                           max_depth=10,
                           learning_rate=0.01,
                           n_estimators=2000,
                           subsample=0.8,
                           feature_fraction=0.8,
                           reg_alpha=0.5,
                           reg_lambda=0.5,
                           random_state=2022,
                           metric='auc',
                           boosting_type='gbdt',
                           subsample_freq=1,
                           bagging_fraction=0.8)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
prob = []
mean_acc = 0
for k,(train_index, test_index) in enumerate(skf.split(X, y)):
    print(k)
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    # 训练
    print(y_val)
    model = model.fit(X_train,
                          y_train,
                          eval_set=[(X_val, y_val)],
                          eval_metric='auc',
                          verbose = True)
    # 正式预测
    test_y_pred = model.predict_proba(test_features)
    prob.append(test_y_pred)

0
0        0
21       0
22       1
26       1
29       1
        ..
49375    0
49384    1
49389    0
49396    0
49399    0
Name: label, Length: 9826, dtype: int64
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[1]	valid_0's auc: 0.752973
[2]	valid_0's auc: 0.773637
[3]	valid_0's auc: 0.780437
[4]	valid_0's auc: 0.78273
[5]	valid_0's auc: 0.785849
[6]	valid_0's auc: 0.787501
[7]	valid_0's auc: 0.789968
[8]	valid_0's auc: 0.790381
[9]	valid_0's auc: 0.791125
[10]	valid_0's auc: 0.791925
[11]	valid_0's auc: 0.791945
[12]	valid_0's auc: 0.791614
[13]	valid_0's auc: 0.791177
[14]	valid_0's auc: 0.792351
[15]	valid_0's auc: 0.793195
[16]	valid_0's auc: 0.793089
[17]	valid_0's auc: 0.793794
[18]	valid_0's auc: 0.794211
[19]	valid_0's auc: 0.794348
[20]	valid_0's auc: 0.794608
[21]	valid_0's auc: 0.7948

[293]	valid_0's auc: 0.805968
[294]	valid_0's auc: 0.806016
[295]	valid_0's auc: 0.80603
[296]	valid_0's auc: 0.806054
[297]	valid_0's auc: 0.806112
[298]	valid_0's auc: 0.806101
[299]	valid_0's auc: 0.806112
[300]	valid_0's auc: 0.806173
[301]	valid_0's auc: 0.806208
[302]	valid_0's auc: 0.80623
[303]	valid_0's auc: 0.806267
[304]	valid_0's auc: 0.806314
[305]	valid_0's auc: 0.806285
[306]	valid_0's auc: 0.806341
[307]	valid_0's auc: 0.806359
[308]	valid_0's auc: 0.806378
[309]	valid_0's auc: 0.8064
[310]	valid_0's auc: 0.806402
[311]	valid_0's auc: 0.806441
[312]	valid_0's auc: 0.806454
[313]	valid_0's auc: 0.806463
[314]	valid_0's auc: 0.80651
[315]	valid_0's auc: 0.806536
[316]	valid_0's auc: 0.806553
[317]	valid_0's auc: 0.806572
[318]	valid_0's auc: 0.806638
[319]	valid_0's auc: 0.806678
[320]	valid_0's auc: 0.806736
[321]	valid_0's auc: 0.806763
[322]	valid_0's auc: 0.806779
[323]	valid_0's auc: 0.8068
[324]	valid_0's auc: 0.806816
[325]	valid_0's auc: 0.806832
[326]	valid_0's a

[576]	valid_0's auc: 0.810335
[577]	valid_0's auc: 0.810339
[578]	valid_0's auc: 0.810332
[579]	valid_0's auc: 0.810329
[580]	valid_0's auc: 0.810337
[581]	valid_0's auc: 0.810354
[582]	valid_0's auc: 0.810353
[583]	valid_0's auc: 0.810354
[584]	valid_0's auc: 0.810372
[585]	valid_0's auc: 0.810358
[586]	valid_0's auc: 0.810366
[587]	valid_0's auc: 0.81037
[588]	valid_0's auc: 0.810391
[589]	valid_0's auc: 0.810409
[590]	valid_0's auc: 0.810405
[591]	valid_0's auc: 0.810403
[592]	valid_0's auc: 0.810434
[593]	valid_0's auc: 0.810419
[594]	valid_0's auc: 0.810437
[595]	valid_0's auc: 0.810453
[596]	valid_0's auc: 0.810453
[597]	valid_0's auc: 0.810504
[598]	valid_0's auc: 0.810489
[599]	valid_0's auc: 0.810512
[600]	valid_0's auc: 0.810515
[601]	valid_0's auc: 0.810529
[602]	valid_0's auc: 0.810561
[603]	valid_0's auc: 0.810598
[604]	valid_0's auc: 0.810603
[605]	valid_0's auc: 0.810611
[606]	valid_0's auc: 0.810646
[607]	valid_0's auc: 0.810675
[608]	valid_0's auc: 0.81069
[609]	valid_

[888]	valid_0's auc: 0.812364
[889]	valid_0's auc: 0.812348
[890]	valid_0's auc: 0.812356
[891]	valid_0's auc: 0.812352
[892]	valid_0's auc: 0.812345
[893]	valid_0's auc: 0.812347
[894]	valid_0's auc: 0.81234
[895]	valid_0's auc: 0.812357
[896]	valid_0's auc: 0.812355
[897]	valid_0's auc: 0.812363
[898]	valid_0's auc: 0.812338
[899]	valid_0's auc: 0.812343
[900]	valid_0's auc: 0.81235
[901]	valid_0's auc: 0.812354
[902]	valid_0's auc: 0.812379
[903]	valid_0's auc: 0.812397
[904]	valid_0's auc: 0.812392
[905]	valid_0's auc: 0.81239
[906]	valid_0's auc: 0.81237
[907]	valid_0's auc: 0.812356
[908]	valid_0's auc: 0.812394
[909]	valid_0's auc: 0.812411
[910]	valid_0's auc: 0.812397
[911]	valid_0's auc: 0.812383
[912]	valid_0's auc: 0.812384
[913]	valid_0's auc: 0.812387
[914]	valid_0's auc: 0.812369
[915]	valid_0's auc: 0.812382
[916]	valid_0's auc: 0.812384
[917]	valid_0's auc: 0.812398
[918]	valid_0's auc: 0.812418
[919]	valid_0's auc: 0.812416
[920]	valid_0's auc: 0.812419
[921]	valid_0'

[1188]	valid_0's auc: 0.813376
[1189]	valid_0's auc: 0.81338
[1190]	valid_0's auc: 0.813369
[1191]	valid_0's auc: 0.813382
[1192]	valid_0's auc: 0.813378
[1193]	valid_0's auc: 0.813393
[1194]	valid_0's auc: 0.813403
[1195]	valid_0's auc: 0.813387
[1196]	valid_0's auc: 0.813373
[1197]	valid_0's auc: 0.813387
[1198]	valid_0's auc: 0.813396
[1199]	valid_0's auc: 0.813403
[1200]	valid_0's auc: 0.813413
[1201]	valid_0's auc: 0.813413
[1202]	valid_0's auc: 0.813426
[1203]	valid_0's auc: 0.813405
[1204]	valid_0's auc: 0.813412
[1205]	valid_0's auc: 0.813415
[1206]	valid_0's auc: 0.813426
[1207]	valid_0's auc: 0.81345
[1208]	valid_0's auc: 0.813461
[1209]	valid_0's auc: 0.813463
[1210]	valid_0's auc: 0.813485
[1211]	valid_0's auc: 0.813476
[1212]	valid_0's auc: 0.81348
[1213]	valid_0's auc: 0.813459
[1214]	valid_0's auc: 0.813492
[1215]	valid_0's auc: 0.813491
[1216]	valid_0's auc: 0.81349
[1217]	valid_0's auc: 0.813512
[1218]	valid_0's auc: 0.813521
[1219]	valid_0's auc: 0.81352
[1220]	valid_

...

[680]	valid_0's auc: 0.809737
[681]	valid_0's auc: 0.809749
[682]	valid_0's auc: 0.809739
[683]	valid_0's auc: 0.809724
[684]	valid_0's auc: 0.80973
[685]	valid_0's auc: 0.809731
[686]	valid_0's auc: 0.809742
[687]	valid_0's auc: 0.809745
[688]	valid_0's auc: 0.809725
[689]	valid_0's auc: 0.80974
[690]	valid_0's auc: 0.809773
[691]	valid_0's auc: 0.809781
[692]	valid_0's auc: 0.809771
[693]	valid_0's auc: 0.80975
[694]	valid_0's auc: 0.809758
[695]	valid_0's auc: 0.80975
[696]	valid_0's auc: 0.80976
[697]	valid_0's auc: 0.809743
[698]	valid_0's auc: 0.809751
[699]	valid_0's auc: 0.809751
[700]	valid_0's auc: 0.809761
[701]	valid_0's auc: 0.80976
[702]	valid_0's auc: 0.809746
[703]	valid_0's auc: 0.809724
[704]	valid_0's auc: 0.809735
[705]	valid_0's auc: 0.809726
[706]	valid_0's auc: 0.80973
[707]	valid_0's auc: 0.809746
[708]	valid_0's auc: 0.809747
[709]	valid_0's auc: 0.809734
[710]	valid_0's auc: 0.809751
[711]	valid_0's auc: 0.809769
[712]	valid_0's auc: 0.809787
[713]	valid_0's a

[1001]	valid_0's auc: 0.811317
[1002]	valid_0's auc: 0.81132
[1003]	valid_0's auc: 0.811312
[1004]	valid_0's auc: 0.811309
[1005]	valid_0's auc: 0.811325
[1006]	valid_0's auc: 0.811342
[1007]	valid_0's auc: 0.811324
[1008]	valid_0's auc: 0.811309
[1009]	valid_0's auc: 0.811319
[1010]	valid_0's auc: 0.811332
[1011]	valid_0's auc: 0.811348
[1012]	valid_0's auc: 0.811344
[1013]	valid_0's auc: 0.811367
[1014]	valid_0's auc: 0.811366
[1015]	valid_0's auc: 0.811346
[1016]	valid_0's auc: 0.81135
[1017]	valid_0's auc: 0.811347
[1018]	valid_0's auc: 0.811369
[1019]	valid_0's auc: 0.811361
[1020]	valid_0's auc: 0.811349
[1021]	valid_0's auc: 0.81134
[1022]	valid_0's auc: 0.811336
[1023]	valid_0's auc: 0.811338
[1024]	valid_0's auc: 0.811331
[1025]	valid_0's auc: 0.811322
[1026]	valid_0's auc: 0.811329
[1027]	valid_0's auc: 0.811337
[1028]	valid_0's auc: 0.811337
[1029]	valid_0's auc: 0.81134
[1030]	valid_0's auc: 0.811366
[1031]	valid_0's auc: 0.811367
[1032]	valid_0's auc: 0.811362
[1033]	valid

[1321]	valid_0's auc: 0.811861
[1322]	valid_0's auc: 0.81187
[1323]	valid_0's auc: 0.811871
[1324]	valid_0's auc: 0.81189
[1325]	valid_0's auc: 0.811903
[1326]	valid_0's auc: 0.811901
[1327]	valid_0's auc: 0.811898
[1328]	valid_0's auc: 0.811877
[1329]	valid_0's auc: 0.811892
[1330]	valid_0's auc: 0.811899
[1331]	valid_0's auc: 0.81188
[1332]	valid_0's auc: 0.811883
[1333]	valid_0's auc: 0.811896
[1334]	valid_0's auc: 0.811919
[1335]	valid_0's auc: 0.811927
[1336]	valid_0's auc: 0.811932
[1337]	valid_0's auc: 0.811925
[1338]	valid_0's auc: 0.811927
[1339]	valid_0's auc: 0.811935
[1340]	valid_0's auc: 0.811944
[1341]	valid_0's auc: 0.811947
[1342]	valid_0's auc: 0.811953
[1343]	valid_0's auc: 0.811946
[1344]	valid_0's auc: 0.811952
[1345]	valid_0's auc: 0.811943
[1346]	valid_0's auc: 0.811955
[1347]	valid_0's auc: 0.811958
[1348]	valid_0's auc: 0.81197
[1349]	valid_0's auc: 0.811967
[1350]	valid_0's auc: 0.811966
[1351]	valid_0's auc: 0.81199
[1352]	valid_0's auc: 0.812009
[1353]	valid_

[1642]	valid_0's auc: 0.812191
[1643]	valid_0's auc: 0.812204
[1644]	valid_0's auc: 0.812206
[1645]	valid_0's auc: 0.812218
[1646]	valid_0's auc: 0.812195
[1647]	valid_0's auc: 0.812199
[1648]	valid_0's auc: 0.812193
[1649]	valid_0's auc: 0.812222
[1650]	valid_0's auc: 0.812244
[1651]	valid_0's auc: 0.812241
[1652]	valid_0's auc: 0.812241
[1653]	valid_0's auc: 0.812241
[1654]	valid_0's auc: 0.812241
[1655]	valid_0's auc: 0.812241
[1656]	valid_0's auc: 0.81226
[1657]	valid_0's auc: 0.812245
[1658]	valid_0's auc: 0.812243
[1659]	valid_0's auc: 0.812249
[1660]	valid_0's auc: 0.812259
[1661]	valid_0's auc: 0.812264
[1662]	valid_0's auc: 0.812278
[1663]	valid_0's auc: 0.812286
[1664]	valid_0's auc: 0.812259
[1665]	valid_0's auc: 0.812264
[1666]	valid_0's auc: 0.812252
[1667]	valid_0's auc: 0.812229
[1668]	valid_0's auc: 0.81225
[1669]	valid_0's auc: 0.812246
[1670]	valid_0's auc: 0.812247
[1671]	valid_0's auc: 0.81224
[1672]	valid_0's auc: 0.812233
[1673]	valid_0's auc: 0.812235
[1674]	vali

[1960]	valid_0's auc: 0.812256
[1961]	valid_0's auc: 0.812259
[1962]	valid_0's auc: 0.812273
[1963]	valid_0's auc: 0.812267
[1964]	valid_0's auc: 0.812285
[1965]	valid_0's auc: 0.812287
[1966]	valid_0's auc: 0.812281
[1967]	valid_0's auc: 0.812307
[1968]	valid_0's auc: 0.81229
[1969]	valid_0's auc: 0.81229
[1970]	valid_0's auc: 0.812289
[1971]	valid_0's auc: 0.812292
[1972]	valid_0's auc: 0.812278
[1973]	valid_0's auc: 0.812281
[1974]	valid_0's auc: 0.812296
[1975]	valid_0's auc: 0.812291
[1976]	valid_0's auc: 0.812286
[1977]	valid_0's auc: 0.812275
[1978]	valid_0's auc: 0.812278
[1979]	valid_0's auc: 0.812273
[1980]	valid_0's auc: 0.812274
[1981]	valid_0's auc: 0.812285
[1982]	valid_0's auc: 0.812302
[1983]	valid_0's auc: 0.812293
[1984]	valid_0's auc: 0.812322
[1985]	valid_0's auc: 0.812306
[1986]	valid_0's auc: 0.812316
[1987]	valid_0's auc: 0.812321
[1988]	valid_0's auc: 0.812322
[1989]	valid_0's auc: 0.812343
[1990]	valid_0's auc: 0.812329
[1991]	valid_0's auc: 0.812327
[1992]	val

In [104]:
# lgb结果
new_result = (prob[0]+prob[1]+prob[2]+prob[3]+prob[4])/5
final_result = []
for i in new_result:
    if i[0]>0.5:
        final_result.append(0)
    else:
        final_result.append(1)
        
res = pd.DataFrame()
res['prediction'] = final_result
res['index'] = res.index
res.to_csv('./result/paws-x-zh.tsv', index=False, sep='\t')
res

,prediction,index
0,1,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
1995,0,1995
1996,0,1996
1997,0,1997
1998,0,1998
